La siguiente clase nos permite realizar la transcrpcion de un archivo de audio a texto utilizando el API de Open AI, especificando el modelo de reconocimiento de voz Wishper

en el metodo __init__ inicializamos el atriburo transcription de nuestra clase, esot para poder reutilizar una vez se cree.
el metodo transcript_audio recibe un parametro de audio_path que representa la ruta del archivo de audio a transcribir, se verifica si ya existe un trancripcion, si es None, procede con la transcripcion del archivo, lo abre en modo binario ("rb"), ya que el API espera un archivo de este tipo.
llama.

Llamada a la API de OpenAI:
Usa la instancia client de la clase OpenAI para hacer una llamada al método client.audio.transcriptions.create.
Se especifica el modelo whisper-1 para la transcripción de audio.
El archivo de audio se pasa como parámetro (file=audio_file) para que el modelo procese el audio y genere la transcripción.

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="xxs")

class Transcriber:
  def __init__(self):
    self.transcription = None

  def transcript_audio(self, audio_path):
    if self.transcription is None:
      audio_file= open(audio_path, "rb")
      self.transcription = client.audio.transcriptions.create(
        model="whisper-1", 
        file=audio_file
      )
    return self.transcription

La clase Model contiene un metodo llamado call_model el cual recibe una pregunta y el contexto para generar una respuesta utilizando un modelo determinado, en nuestro caso gpt-4o. ademas de un promt donde especificamos un role, un contenido el cual es una forma de decir como queremos que se comporte, todo esto lo asignamos a la variable completion la cual recibe el resultado de una llamada a la API de OpenAI para generar un "completion" o respuesta la cual retornamos.


In [ ]:
from openai import OpenAI

client = OpenAI(api_key="xxs")

class Model:
  def call_model(self, question, context):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": f"""
          Eres un asistente que ayuda a responder preguntas basado solamente en el siguiente contexto {context}, si no sabes la respuesta dí no lo se,
          sé preciso y conciso en tus respuestas.
        """},
        {"role": "user", "content": question},
      ]
    )
    return completion.choices[0].message.content

la interfaz de usuario que elegimos fue streamlit, ya que es una framework que veniamnos trabajando en otros proyectos, para lo cual creamos un archivo index.py para despligue de la interfaz de la aplicacion, manejando diferente variables de estado para ver el historial del chat, para guardar el contexto que genera nuestro modelo y la validacion de activacion de chat una vez se selecciones el archivo de audio o video para procesar la transcripcion.

In [ ]:
import streamlit as st
import os
from openai import OpenAI
import streamlit as st

import Transcriber
import Model

# Configuración inicial de la interfaz de usuario
st.set_page_config(layout="wide")
#instanciamos nuestro modelo de transcripcion donde usamos wishper
transcriber_model = Transcriber.Transcriber()
#instanciamos el modelo que se encargara de contestar las preguntas gpt-4o
chat_model = Model.Model()
#creamos un sidebar para ingresar aqui los componentes necesarios para iniciar el proceso. 
with st.sidebar:
    st.title("Procesador de archivos de Audio/Video")
    #creamos caja de texto donde se solicita el path o url del audio a procesar y lo asignamos a la variable
    file_url = st.text_input("Ingresa la URL del archivo de audio o video")

#se crea boton y se valida lo necesario para procesar
    if st.button("Process") and file_url:
        try:
            #se llama al metodo trancribir audio de nuestra clase transcriber y se guarda en una variable
            transcription = transcriber_model.transcript_audio(file_url)
            print("*** procesar")
            #se crea variable de contexto la cual almacena la transcrpcion de nuestro audio
            st.session_state.contexto=transcription.text
            st.write(st.session_state.contexto)
            #bandera para habilitar el chat
            st.session_state.chat_enabled = True
        except Exception as e:
            st.error(f"Error al procesar el archivo desde la URL: {e}")
            st.session_state.chat_enabled = False


# Verificar si el chat está habilitado
if st.session_state.get("chat_enabled", False):
    # Área de chat en la pantalla principal
    st.header("Chat")
    if "chat_history" not in st.session_state:
        #variable de sesion para manejo de historial del chat
        st.session_state.chat_history = []


    # Crear el campo de entrada de texto chat
    user_input = st.chat_input("Escribe tu mensaje:")
    if user_input:
        if user_input:
            # Añadimos el mensaje del usuario al historial del chat
            st.session_state.chat_history.append(f"Usuario: {user_input}")
            print(user_input)
            print("*******"+st.session_state.contexto)
            # Generamos una respuesta usando el modelo de chat
            response = chat_model.call_model(user_input,st.session_state.contexto)
            #se muestra respuesta del modelo en el chat
            st.session_state.chat_history.append(f"IA: {response}")
#se imprime histoprial del chat
    for message in st.session_state.chat_history:
        st.write(message)
else:
    st.info("Por favor, procesa un archivo para habilitar el chat.")

A continuacion una explicacion de la arquitectura general de nuestra aplicacion culla  arquitectura es modular, ya que separa claramente el procesamiento de audio, la gestión de la interfaz de usuario y la generación de respuestas, lo que permite extender o modificar cada componente sin afectar a los demás.

### 1. **Capa de Procesamiento de Audio/Video**
   - **Módulo `Transcriber`**: Esta parte del sistema utiliza `Transcriber.Transcriber()` para realizar la transcripción de archivos de audio o video. Al proporcionar la URL de un archivo, el código llama a `transcriber_model.transcript_audio(file_url)`, lo que transforma el contenido hablado en texto.
   - **Propósito**: La transcripción generada es el "contexto" que se usará como referencia para la conversación en el chat. Este contexto se guarda en `st.session_state.contexto`.

### 2. **Capa de Gestión de Contexto e Interfaz de Chat**
   - La interfaz de usuario (UI) se maneja a través de Streamlit, permitiendo a los usuarios proporcionar la URL del archivo, iniciar el procesamiento y participar en el chat si el procesamiento es exitoso.
   - **Manejo del Estado**: El código usa `st.session_state` para almacenar el contexto de la transcripción y los mensajes del historial de chat.
   - **Función de Chat**: Al activar el chat, el usuario puede enviar preguntas o mensajes. Cada mensaje se procesa y se guarda junto con las respuestas de la IA, que se basan en el contexto transcrito.

### 3. **Capa del Modelo de Lenguaje (Model.Model)**
   - **Módulo `Model`**: `Model.Model()` es la instancia que representa el modelo de lenguaje, probablemente basado en una arquitectura de Transformer. Este modelo genera respuestas contextuales usando el historial de la conversación junto con la transcripción inicial.
   - **Transformers**: Los transformers están formados por capas de atención y feed-forward, permitiendo al modelo comprender dependencias de largo alcance en el texto de entrada y responder de forma coherente al contexto de la transcripción.
   - **Llamada al modelo**: Cada vez que el usuario envía un mensaje, el código llama a `chat_model.call_model(user_input, st.session_state.contexto)`, que usa el mensaje y el contexto transcrito para generar una respuesta relevante.

### Diagrama de Flujo de la Arquitectura

Aquí tienes una representación gráfica simple de esta arquitectura:

```plaintext
          ┌────────────────────────────────────────┐
          │       Procesamiento de Audio/Video     │
          │         (Transcriber.Transcriber)      │
          │  Entrada: Archivo de audio/video (URL) │
          │  Salida: Texto transcrito (contexto)   │
          └────────────────────────────────────────┘
                             │
                             │ Contexto (texto)
                             ▼
          ┌────────────────────────────────────────┐
          │           Gestión de la Interfaz       │
          │          e Interacción del Chat        │
          │                (Streamlit)             │
          │   ───> Entrada del usuario (mensaje)   │
          │   ───> Historial del chat              │
          └────────────────────────────────────────┘
                             │
                             │ Entrada del usuario
                             ▼
          ┌────────────────────────────────────────┐
          │      Modelo de Lenguaje (Transformers) │
          │           (Model.Model)                │
          │     Entrada: mensaje + contexto        │
          │     Salida: Respuesta del modelo       │
          └────────────────────────────────────────┘
```

En este diagrama:
- **Procesamiento de Audio/Video** se refiere a la transcripción inicial del archivo.
- **Gestión de la Interfaz** es el código de Streamlit para la interacción con el usuario.
- **Modelo de Lenguaje** es el Transformer que maneja las consultas contextuales del chat.


